In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple, Structural } from "recursive-set";
import { RegExp } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";
import { nfa2dfa, Char } from "./01-NFA-2-DFA";
import { RegExp2NFA } from "./03-RegExp-2-NFA";
import { minimize } from "./07-Minimize";
import { fsm_complement, findWitness, regExpEquiv } from "./09-Equivalence";
import { dfa2dot, renderLegend, renderComparisonLayout } from "./FSM-2-Dot";
const viz = await instance();

# Test the Equivalence of Regular Expressions

In this notebook, we test the equivalence algorithm implemented in `09-Equivalence.ipynb`.

In [ ]:
function test(Σ: RecursiveSet<Char>, s1: string, s2: string) {
    const r1 = parse(s1), r2 = parse(s2);
    if (regExpEquiv(r1, r2, Σ)) {
        console.log(`The regular expressions ${s1} and ${s2} are equivalent.`);
    } else {
        console.log(`The regular expressions ${s1} and ${s2} are not equivalent.`);
    }
}

We define the alphabet $\Sigma = \{a, b, c\}$ and run several test cases. These examples range from simple identities to complex structural variations.

In [ ]:
const Sigma = new RecursiveSet<Char>("a", "b", "c");

In [ ]:
test(Sigma, '(ε+a)(a+ε)*(a+ε)', 'a*');

In [ ]:
test(Sigma, "(ba)(ba)*", "b(ab)*a");

In [ ]:
test(Sigma, "(a+b+c)*(ac*b+bc*a)(a+b+c)*", "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*");

In [ ]:
test(
    Sigma,
    "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)",
    "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",
);

In [ ]:
test(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");

### Visual Comparison Pipeline

The function `testVisual` orchestrates the entire verification process. It serves as a high-level driver that combines parsing, automata construction, minimization, equivalence checking, and visualization.

**The Workflow:**

1.  **Parse & Convert:** The helper `toDFA` converts the regular expression strings into DFAs (Regex $\to$ NFA $\to$ DFA). 
2.  **Minimization:** We apply `minimize` to both automata.
    * *Theoretical Note:* Two regular expressions are equivalent **if and only if** their minimal DFAs are isomorphic (structurally identical up to state renaming). This makes the visual comparison meaningful.
3.  **Equivalence Check:** We compute the difference languages using `fsm_complement` (which supports our generic/minimized automata) and search for **witnesses** using `findWitness`.
4.  **Rendering:** Finally, we generate SVG diagrams and an HTML legend to display the two minimal machines side-by-side for manual inspection.

In [ ]:
const toMinDFA = (s: string, Σ: RecursiveSet<Char>) => 
    minimize(nfa2dfa(new RegExp2NFA(Σ).toNFA(parse(s))));

async function testVisual(Σ: RecursiveSet<Char>, s1: string, s2: string) {
    console.log(`\n🔎 Inspecting: "${s1}" vs "${s2}"`);
    try {
        const M1 = toMinDFA(s1, Σ);
        const M2 = toMinDFA(s2, Σ);
        const w1 = findWitness(fsm_complement(M1, M2));
        const w2 = findWitness(fsm_complement(M2, M1));
        if (!w1 && !w2) {
            console.log("✅ RESULT: Equivalent! (Isomorphic Minimal DFAs)");
        } else {
            console.log("❌ RESULT: NOT Equivalent.");
            if (w1) console.log(`   ➡ Witness 1 (in L1 \\ L2): "${w1}"`);
            if (w2) console.log(`   ➡ Witness 2 (in L2 \\ L1): "${w2}"`);
        }
        const svg = (M: ReturnType<typeof minimize>) => 
            viz.renderString(dfa2dot(M), { format: "svg" });
        display.html(renderComparisonLayout(
            s1, svg(M1), renderLegend(M1),
            s2, svg(M2), renderLegend(M2)
        ));
    } catch (e) { console.error(e); }
}

In [ ]:
await testVisual(Sigma, "(ε+a)(a+ε)*(a+ε)", "a*"); 

In [ ]:
await testVisual(Sigma, "(ba)(ba)*", "b(ab)*a"); 

In [ ]:
await testVisual(Sigma, "(a+b+c)*(ac*b+bc*a)(a+b+c)*", "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*"); 

In [ ]:
await testVisual(Sigma, "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)", "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",); 

In [ ]:
await testVisual(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*"); 